# JPX Tokyo Stock Exchange Prediction

## Competition

### Introduction
This competition is meant to define a model to find make accurate predictionas for the Japanese Stock Trade.

**stock_prices.csv** The core file of interest. Includes the daily closing price for each stock and the target column.

**options.csv** Data on the status of a variety of options based on the broader market. Many options include implicit predictions of the future price of the stock market and so may be of interest even though the options are not scored directly.

**secondary_stock_prices.csv** The core dataset contains on the 2,000 most commonly traded equities but many less liquid securities are also traded on the Tokyo market. This file contains data for those securities, which aren't scored but may be of interest for assessing the market as a whole.

**trades.csv** Aggregated summary of trading volumes from the previous business week.

**financials.csv** Results from quarterly earnings reports.

**stock_list.csv** - Mapping between the SecuritiesCode and company names, plus general information about which industry the company is in.

## Data Preparation Stage